In [2]:
import pandas as pd
import sqlite3
import warnings
import re
from datetime import datetime
warnings.filterwarnings(action='ignore')

In [3]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [4]:
con = sqlite3.connect("../db/hr.db")
df = pd.read_sql_query("SELECT 자료생성년월, 사업장명, 사업자등록번호, 사업장업종코드명, 가입자수, 신규취득자수, 상실가입자수, 당월고지금액  from pension", con)

In [5]:
df['자료생성년월'].unique()

array(['2023-01', '2022-12', '2022-11', '2022-10', '2022-09', '2022-08',
       'Jul-22', 'Jun-22', 'May-22', 'Apr-22', 'Mar-22', 'Feb-22',
       'Jan-22', '2023-02'], dtype=object)

In [6]:
df1 = df.copy()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7636766 entries, 0 to 7636765
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   자료생성년월    object
 1   사업장명      object
 2   사업자등록번호   int64 
 3   사업장업종코드명  object
 4   가입자수      int64 
 5   신규취득자수    int64 
 6   상실가입자수    int64 
dtypes: int64(4), object(3)
memory usage: 407.8+ MB


In [24]:
df1["자료생성년월"].unique()

array(['2023-01', '2022-12', '2022-11', '2022-10', '2022-09', '2022-08',
       'Jul-22', 'Jun-22', 'May-22', 'Apr-22', 'Mar-22', 'Feb-22',
       'Jan-22'], dtype=object)

In [25]:
df1 = df1[df1["자료생성년월"] != "2023-01"]
df1["자료생성년월"].unique()

array(['2022-12', '2022-11', '2022-10', '2022-09', '2022-08', 'Jul-22',
       'Jun-22', 'May-22', 'Apr-22', 'Mar-22', 'Feb-22', 'Jan-22'],
      dtype=object)

In [26]:
gdf = df1.groupby(['사업장업종코드명'])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index()
gdf

,사업장업종코드명,가입자수,신규취득자수,상실가입자수
0,1차 금속제품 도매업,208591,6147,5103
1,1차 유리제품 유리섬유 및 광학용 유리 제조업,89,2,3
2,1차 유리제품 유리섬유 및 광학용 유리 제조업,998,30,25
3,BIZ_NO미존재사업장,14378363,563756,559962
4,가공 및 정제염 제조업,3811,107,88
...,...,...,...,...
1335,화훼류 및 식물 도매업,20804,1004,912
1336,화훼작물 재배업,561,70,41
1337,환경 관련 엔지니어링 서비스업,16600,960,541
1338,환경설비 건설업,1343,66,54


In [ ]:
# gdf["사업장업종코드명"].unique().tolist()

In [27]:
gdf[gdf["사업장업종코드명"] == '건설 및 채광용 기계장비 제조업']

,사업장업종코드명,가입자수,신규취득자수,상실가입자수
54,건설 및 채광용 기계장비 제조업,112227,2809,2471


In [28]:
gdf[gdf["사업장업종코드명"] == '화물 자동차 및 특수 목적용 자동차 제조업']

,사업장업종코드명,가입자수,신규취득자수,상실가입자수
1320,화물 자동차 및 특수 목적용 자동차 제조업,1654350,15176,15832


In [37]:
def 월평균가입자수(가입자수):
    return int(가입자수/12)

In [38]:
df2 = df1[df1["사업장업종코드명"] == '건설 및 채광용 기계장비 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
df2 = df2.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
df2["월평균가입자수"] = df2["가입자수"].apply(월평균가입자수)
df2["상실률"] = df2["상실가입자수"]/df2["월평균가입자수"]
df2

,사업장명,가입자수,신규취득자수,상실가입자수,월평균가입자수,상실률
445,현대두산인프라코어(주),32434,369,517,2702,0.191340
443,현대건설기계(주),16192,172,234,1349,0.173462
447,현대제뉴인 주식회사,5135,424,57,427,0.133489
86,(주)진성티이씨,2690,39,32,224,0.142857
319,주식회사세안정기,2495,53,55,207,0.265700
428,캐터필라정밀씰 주식회사,2039,18,19,169,0.112426
48,(주)수산중공업,1839,47,29,153,0.189542
152,대모엔지니어링(주),1197,10,15,99,0.151515
349,주식회사영인,1124,119,24,93,0.258065
142,다보정밀(주),940,30,18,78,0.230769


In [39]:
df3 = df1[df1["사업장업종코드명"] == '화물 자동차 및 특수 목적용 자동차 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
df3 = df3.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
df3["월평균가입자수"] = df3["가입자수"].apply(월평균가입자수)
df3["상실률"] = df3["상실가입자수"]/df3["월평균가입자수"]
df3

,사업장명,가입자수,신규취득자수,상실가입자수,월평균가입자수,상실률
355,현대자동차(주),815960,7250,7378,67996,0.108506
103,기아(주)AutoLand화성,148098,765,754,12341,0.061097
353,현대모비스(주),132499,1467,848,11041,0.076805
350,한국지엠주식회사,101318,393,594,8443,0.070354
99,기아(주) AutoLand광주,82805,536,636,6900,0.092174
98,기아 주식회사,68550,566,391,5712,0.068452
102,기아(주)AutoLand광명,63055,658,886,5254,0.168633
157,쌍용자동차(주),51624,20,212,4302,0.049279
129,르노코리아자동차 주식회사,38119,926,880,3176,0.277078
100,기아(주) 영업본부,31511,1,136,2625,0.051810


In [ ]:
비디오 및 기타 영상 기기 제조업


In [43]:
df4 = df1[df1["사업장업종코드명"] == '비디오 및 기타 영상 기기 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
df4 = df4.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
df4["월평균가입자수"] = df4["가입자수"].apply(월평균가입자수)
df4["상실률"] = df4["상실가입자수"]/df4["월평균가입자수"]
df4.head(20)

,사업장명,가입자수,신규취득자수,상실가입자수,월평균가입자수,상실률
278,삼성전자(주),1377173,12957,6189,114764,0.053928
276,삼성디스플레이(주),259247,1299,1784,21603,0.082581
753,에이치피프린팅코리아 유한회사,12152,32,266,1012,0.262846
720,서울반도체(주),5208,101,218,434,0.502304
252,동양이엔피(주),4527,63,88,377,0.233422
32,(주)대성엘텍,3688,57,142,307,0.462541
211,(주)케이원전자,3365,128,118,280,0.421429
1032,크레신(주),2791,42,68,232,0.293103
213,(주)코멧,2406,63,92,200,0.460000
1035,티제이미디어주식회사,2296,57,33,191,0.172775


In [44]:
df5 = df1[df1["사업장업종코드명"] == '컴퓨터 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
df5 = df5.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
df5["월평균가입자수"] = df5["가입자수"].apply(월평균가입자수)
df5["상실률"] = df5["상실가입자수"]/df5["월평균가입자수"]
df5.head(20)

,사업장명,가입자수,신규취득자수,상실가입자수,월평균가입자수,상실률
494,에스케이하이닉스 주식회사,361531,4021,2224,30127,0.073821
442,삼성전기(주),148376,1697,1136,12364,0.091880
766,주식회사이녹스첨단소재,8517,150,122,709,0.172073
878,티맥스소프트(주),5820,172,137,485,0.282474
176,(주)에스티아이,4647,115,69,387,0.178295
880,파이오링크(주),4450,119,75,370,0.202703
499,에이치엔아이엔씨(주),4167,107,356,347,1.025937
572,주식회사 링네트,3132,81,78,261,0.298851
418,디아이티(주),3086,44,91,257,0.354086
338,(주)퓨트로닉,2995,101,98,249,0.393574


In [45]:
df6 = df1[df1["사업장업종코드명"] == '산업용 로봇 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
df6 = df6.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
df6["월평균가입자수"] = df6["가입자수"].apply(월평균가입자수)
df6["상실률"] = df6["상실가입자수"]/df6["월평균가입자수"]
df6.head(20)

,사업장명,가입자수,신규취득자수,상실가입자수,월평균가입자수,상실률
647,이오테크닉스,7709,102,69,642,0.107477
450,두산산업차량주식회사,7239,179,105,603,0.174129
245,(주)우진플라임,6262,148,99,521,0.190019
332,(주)테스,4814,60,41,401,0.102244
1095,한화정밀기계주식회사(본사),4498,91,35,374,0.093583
1063,퍼스텍(주),4232,36,25,352,0.071023
1100,현대로보틱스 주식회사,4166,44,78,347,0.224784
103,(주)미래컴퍼니,3675,106,105,306,0.343137
355,(주)피제이전자,2916,86,62,243,0.255144
598,엠에이티플러스（주）,2860,76,67,238,0.281513


In [47]:
df7 = df1[df1["사업장업종코드명"] == '농업 및 임업용 기계 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
df7 = df7.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
df7["월평균가입자수"] = df7["가입자수"].apply(월평균가입자수)
df7["상실률"] = df7["상실가입자수"]/df7["월평균가입자수"]
df7.head(20)

,사업장명,가입자수,신규취득자수,상실가입자수,월평균가입자수,상실률
19,(주)대동,14487,585,516,1207,0.427506
251,엘에스엠트론(주)하이테크센터,3593,57,48,299,0.160535
305,주식회사 아세아텍,3256,93,76,271,0.280443
320,주식회사 티와이엠,3247,150,51,270,0.188889
140,국제종합기계(주),3226,44,42,268,0.156716
321,주식회사 티와이엠 옥천공장,3165,552,35,263,0.133080
286,주식회사 대동모빌리티,2620,119,16,218,0.073394
52,(주)삼우농기,1597,59,49,133,0.368421
24,(주)대원지에스아이,1291,30,31,107,0.289720
116,(주)태성공업,1244,28,22,103,0.213592
